<a href="https://colab.research.google.com/github/sujeengim/STUDY/blob/main/wandbTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# wandb 설치 및 임포트

In [1]:
pip install wandb

In [2]:
import wandb

In [3]:
wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: skhu202114061 (skhu202114061-college). Use `wandb login --relogin` to force relogin


True

# 모듈 임포트

In [4]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# config 세팅; 학습에 필요한 구성들 그룹화하기

In [5]:
config = {
    'epochs':5,
    'classes':10,
    'batch_size' : 128,
    'kernels':[16,32],
    'weight_decay': 0.0005,
    'learning_rate': 1e-3,
    'dataset':'MNIST',
    'architecture':'CNN',
    'seed': 42,
}

# 함수 작성

In [6]:
# data loader 함수
def make_loader(batch_size, train=True):
    full_dataset = datasets.MNIST(root='./data/MNIST', train=train,
                                  download=True, transform=transforms.ToTensor())
    loader = DataLoader(dataset=full_dataset,
                        batch_size=batch_size,
                        shuffle=True,
                        pin_memory=True, num_workers=2)
    return loader

In [7]:
# model 함수
class ConvNet(nn.Module):
    # 틀만 만들어두기
    def __init__(self, kernels, classes=10):
        super(ConvNet, self).__init__()


        self.layer1 = nn.Sequential(
            nn.Conv2d(1, kernels[0], kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, kernels[1], kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.fc = nn.Linear(7*7*kernels[-1], classes)

    #틀 적용시키기
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(x)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [8]:
#trian 함수
def train(model, loader, criterion, optimizer, device, config):
    cumu_loss = 0
    tqdm_loader = tqdm(loader)
    for images, labels in tqdm_loader:
        images, labels = images.to(device), labels.to(device)
        logit = model(images)
        loss = criterion(logit, labels)
        cumu_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return loss

In [9]:
#valid 함수
def valid(model, test_loader):
    model.eval()
    acc_list = []
    with torch.no_grad():
        tqdm_loader = tqdm(test_loader)
        for images, labels in tqdm_loader:
            logit = model(images)
            pred = torch.argmax(logit, dim=1)
            batch_acc = accuracy_score(labels, pred)
            acc_list.append(batch_acc)
    return sum(acc_list) / len(acc_list)

In [10]:
#run 함수
def run(config=None):
    #wandb web 서버와 연결하기; 프젝이름과 id
    wandb.init(project='test-pytorch', entity='skhu202114061-college', config=config)
    config = wandb.config
    device = "cuda" if torch.cuda.is_available() else "cpu"

    train_loader = make_loader(batch_size = config.batch_size, train=True)
    test_loader  = make_loader(batch_size=config.batch_size, train=False)

    model = ConvNet(config.kernels, config.classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
    wandb.watch(model, criterion, log="all", log_freq=10)

    for epoch in range(1, config.epochs):
        loss = train(model, train_loader, criterion, optimizer, device, config)
        print(f"[Train] Epoch {epoch:02} | Loss: {loss:.2f}")
        wandb.log({"Loss": loss})
        if epoch % config.val_evrey == 0:
            acc = valid(model, test_loader)
            print(f"[valid] Epoch {epoch:02} | Acc: {acc:.3f}")
            wandb.log({"Acc": acc})
    return model

In [11]:
model = run(config)

  0%|          | 0/469 [00:01<?, ?it/s]


RuntimeError: Given groups=1, weight of size [32, 16, 5, 5], expected input[128, 1, 28, 28] to have 16 channels, but got 1 channels instead